In [103]:
from __future__ import print_function
from functools import reduce
import re
import tarfile

import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences


def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen), np.array(ys)

RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)
# Default QA1 with 1000 samples
# challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

vocab = set()
for story, q, answer in train + test:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

x, xq, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)

print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

print('Build model...')

sentence = layers.Input(shape=(story_maxlen,), dtype='int32')
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
encoded_sentence = layers.Dropout(0.3)(encoded_sentence)

question = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question = layers.Dropout(0.3)(encoded_question)
encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question)
encoded_question = layers.RepeatVector(story_maxlen)(encoded_question)

merged = layers.add([encoded_sentence, encoded_question])
merged = RNN(EMBED_HIDDEN_SIZE)(merged)
merged = layers.Dropout(0.3)(merged)
preds = layers.Dense(vocab_size, activation='softmax')(merged)

model = Model([sentence, question], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print('Training')
model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']
x.shape = (10000, 68)
xq.shape = (10000, 4)
y.shape = (10000, 22)
story_maxlen, query_maxlen = 68, 4
Build model...
Training
Train on 9500 samples, validate on 500 samples
Epoch 1/40
9500/9500 [==============================] - 34s 4ms/step - loss: 1.9477 - acc: 0.1827 - val_loss: 1.7059 - val_acc: 0.3180
Epoch 2/40
9500/9500 [==============================] - 32s 3ms/step - loss: 1.4969 - acc: 0.4282 - val_loss: 1.3017 - val_acc: 0.4820
Epoch 3/40
9500/9500 [==============================] - 33s 3ms/step - loss: 1.2703 - acc: 0.5114 - val_loss: 1.2407 - val_acc: 0.4780
Epoch 4/40
9500/9500 [==============================] - 33s 3ms/step - loss: 1.1985 - acc: 0.5168 - val_loss: 1.2056 - val_acc: 0.5240
Epoch 5/40
9500/9500 [==============================] - 32s 3ms/step - loss: 1.1619 - acc

In [39]:
train = get_stories(tar.extractfile(challenge.format('train')))

/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [160]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 4, 50)        1100        input_8[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 68)           0                                            
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 4, 50)        0           embedding_8[0][0]                
__________________________________________________________________________________________________
embedding_

In [7]:
train[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'John',
  'went',
  'to',
  'the',
  'hallway',
  '.'],
 ['Where', 'is', 'Mary', '?'],
 'bathroom')

In [9]:
test[0]

(['John',
  'travelled',
  'to',
  'the',
  'hallway',
  '.',
  'Mary',
  'journeyed',
  'to',
  'the',
  'bathroom',
  '.'],
 ['Where', 'is', 'John', '?'],
 'hallway')

In [10]:
train[0] + test[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'John',
  'went',
  'to',
  'the',
  'hallway',
  '.'],
 ['Where', 'is', 'Mary', '?'],
 'bathroom',
 ['John',
  'travelled',
  'to',
  'the',
  'hallway',
  '.',
  'Mary',
  'journeyed',
  'to',
  'the',
  'bathroom',
  '.'],
 ['Where', 'is', 'John', '?'],
 'hallway')

In [14]:
x[0]#Mary moved to the bathroom.转化为如下形式

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  5, 16, 19, 18,  9,  1,  4, 21, 19, 18, 12,  1], dtype=int32)

In [24]:
vocab[4]#vocab从0开始的，故array中的5变为4

'Mary'

In [25]:
vocab[15]

'moved'

In [26]:
vocab[18]

'to'

In [35]:
xq[0]

array([ 7, 13,  5,  2], dtype=int32)

In [36]:
xq.shape

(10000, 4)

In [104]:
s = np.expand_dims(xq[0], axis=0)

In [105]:
s.shape

(1, 4)

In [106]:
s

array([[ 7, 13,  5,  2]], dtype=int32)

In [113]:
m = np.expand_dims(x[0], axis=0)

In [114]:
m.shape

(1, 68)

In [116]:
a = model.predict([m,s])

In [123]:
y = np.argmax(a, axis = 1)

In [125]:
vocab[y[0] + 1]#测输入Mary moved to the bathroom.John went to the hallway.Where is Mary?产生如下结果

'bathroom'

In [126]:
vocab

['.',
 '?',
 'Daniel',
 'John',
 'Mary',
 'Sandra',
 'Where',
 'back',
 'bathroom',
 'bedroom',
 'garden',
 'hallway',
 'is',
 'journeyed',
 'kitchen',
 'moved',
 'office',
 'the',
 'to',
 'travelled',
 'went']

In [193]:
x1 = "Sandra moved to the garden.Sandra went back to the kitchen."
xq1 = "Where is Sandra?"

In [194]:
x1 = tokenize(x1)
xq1 = tokenize(xq1)

/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [195]:
x1
xq1

['Where', 'is', 'Sandra', '?']

In [196]:
word_idx

{'.': 1,
 '?': 2,
 'Daniel': 3,
 'John': 4,
 'Mary': 5,
 'Sandra': 6,
 'Where': 7,
 'back': 8,
 'bathroom': 9,
 'bedroom': 10,
 'garden': 11,
 'hallway': 12,
 'is': 13,
 'journeyed': 14,
 'kitchen': 15,
 'moved': 16,
 'office': 17,
 'the': 18,
 'to': 19,
 'travelled': 20,
 'went': 21}

In [197]:
x1 = [word_idx[w] for w in x1]

In [198]:
x1

[6, 16, 19, 18, 11, 1, 6, 21, 8, 19, 18, 15, 1]

In [199]:
x1 = pad_sequences([x1], maxlen=story_maxlen)

In [200]:
x1

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  6, 16, 19, 18, 11,  1,  6, 21,  8, 19, 18, 15,  1]], dtype=int32)

In [201]:
xq1 = [word_idx[w] for w in xq1]

In [202]:
xq1

[7, 13, 6, 2]

In [203]:
xq1 = pad_sequences([xq1], maxlen=query_maxlen)

In [204]:
a = model.predict([x1,xq1])

In [205]:
y = np.argmax(a, axis = 1)

In [206]:
vocab[y[0] - 1]#输出预测结果

'kitchen'